In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jun 19 18:29:14 2024

@author: l00416959
"""

import numpy as np
import itertools

cfg_path = '../Dataset0/Dataset0CfgData3.txt'
# inputdata_path = '../Dataset0/Dataset0InputData1.txt'

# func to read in slices
def read_slice_of_file(file_path, start, end):
    with open(file_path, 'r') as file:
        # use itertools.islice to get slices
        slice_lines = list(itertools.islice(file, start, end))
    return slice_lines

# read RoundYCfgDataX.txt 
slice_lines = read_slice_of_file(cfg_path, 1, 6)
info = np.loadtxt(slice_lines)
tol_samp_num = int(info[0])
port_num = int(info[2])
ant_num = int(info[3])
sc_num = int(info[4])

# read RoundYInputDataX. in slices 
H = []
slice_samp_num = 1000   #number of samples
slice_num = int(tol_samp_num / slice_samp_num) #number of slices
for slice_idx in range(slice_num):
    print(slice_idx)
    slice_lines = read_slice_of_file(inputdata_path, slice_idx * slice_samp_num, (slice_idx + 1) * slice_samp_num)
    Htmp = np.loadtxt(slice_lines)
    Htmp = np.reshape(Htmp, (slice_samp_num, 2, sc_num, ant_num, port_num))
    Htmp = Htmp[:, 0, :, :, :] + 1j*Htmp[:, 1, :, :, :]
    Htmp = np.transpose(Htmp, (0,3,2,1))
    
    if np.size(H) == 0:
        H = Htmp
    else:
        H = np.concatenate((H, Htmp), axis=0)

0


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [1]:
import numpy as np
import itertools

H = np.load('../Dataset0/Dataset0InputData3.npy')

In [2]:
###try extract features with pca
import numpy as np


def extract_features(H, n_components=50):
    """
    Extract spatial and frequency features from the complex channel matrix H using Complex PCA.
    
    Parameters:
        H (numpy.ndarray): Complex channel matrix with shape (samples, 64, 2, 408).
        n_components (int): Number of dimensions to reduce to using PCA.
        
    Returns:
        numpy.ndarray: Reduced feature representation (samples, n_components).
    """
    num_samples, num_antennas, num_user_antennas, num_subcarriers = H.shape

    # Step 1: Compute spatial covariance matrix
    spatial_features = []
    for sample in H:
        # Combine user antenna dimensions into a single matrix (64, 816)
        reshaped_sample = sample.reshape(num_antennas, -1)  # Shape: (64, 816)
        # Compute spatial covariance (64x64)
        spatial_cov = np.matmul(reshaped_sample, reshaped_sample.conj().T)
        spatial_features.append(spatial_cov.flatten())  # Flatten into 1D array

    spatial_features = np.array(spatial_features)  # Shape: (samples, 64*64)

    # Step 2: Compute frequency domain features
    frequency_features = []
    for sample in H:
        # Average across antennas and user antennas for each subcarrier (408)
        freq_avg = np.mean(np.abs(sample), axis=(0, 1))  # Shape: (408,)
        frequency_features.append(freq_avg)

    frequency_features = np.array(frequency_features)  # Shape: (samples, 408)

    # Step 3: Combine spatial and frequency features
    combined_features = np.hstack((spatial_features, frequency_features))  # Shape: (samples, 64*64 + 408)

    # Step 4: Apply Complex PCA for dimensionality reduction
    # Compute covariance matrix of the combined features
    cov_matrix = np.cov(combined_features.T)  # Shape: (features, features)
    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
    # Sort by eigenvalues in descending order
    idx = eigenvalues.argsort()[::-1]
    eigenvectors = eigenvectors[:, idx]
    # Select the top n_components eigenvectors
    principal_components = eigenvectors[:, :n_components]  # Shape: (features, n_components)
    # Project data onto the principal components
    reduced_features = np.dot(combined_features, principal_components)  # Shape: (samples, n_components)

    return reduced_features


In [2]:
import numpy as np
import umap

def extract_features_umap(H, n_components=50):
    """
    使用 UMAP 从复数信道矩阵 H 中提取特征并降维。

    参数:
        H (numpy.ndarray): 复数信道矩阵，形状为 (samples, 64, 2, 408)。
        n_components (int): UMAP 降维的目标维度。

    返回:
        numpy.ndarray: 降维后的特征表示，形状为 (samples, n_components)。
    """
    num_samples, num_antennas, num_user_antennas, num_subcarriers = H.shape

    # 步骤 1：计算空间协方差矩阵的幅度和相位特征
    spatial_features = []
    for sample in H:
        reshaped_sample = sample.reshape(num_antennas, -1)  # 形状: (64, 816)
        spatial_cov = np.matmul(reshaped_sample, reshaped_sample.conj().T)  # 计算协方差矩阵 (64, 64)
        amplitude = np.abs(spatial_cov)  # 幅度 (64, 64)
        phase = np.angle(spatial_cov)    # 相位 (64, 64)
        spatial_features.append(np.hstack((amplitude.flatten(), phase.flatten())))  # 展平并合并

    spatial_features = np.array(spatial_features)  # 形状: (samples, 64*64*2)

    # 步骤 2：计算频域的幅度和相位特征
    frequency_features = []
    for sample in H:
        amplitude_avg = np.mean(np.abs(sample), axis=(0, 1))  # 每个子载波的平均幅度 (408,)
        phase_avg = np.mean(np.angle(sample), axis=(0, 1))    # 每个子载波的平均相位 (408,)
        frequency_features.append(np.hstack((amplitude_avg, phase_avg)))  # 合并幅度和相位

    frequency_features = np.array(frequency_features)  # 形状: (samples, 408*2)

    # 步骤 3：合并空间和频率特征
    combined_features = np.hstack((spatial_features, frequency_features))  # 形状: (samples, 64*64*2 + 408*2)

    # 可选：对特征进行标准化（有助于 UMAP 的效果）
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    combined_features = scaler.fit_transform(combined_features)

    # 步骤 4：使用 UMAP 进行降维
    reducer = umap.UMAP(n_components=n_components, random_state=0)
    reduced_features = reducer.fit_transform(combined_features)

    return reduced_features



/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-14 19:22:28.457098: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731583348.578535    2869 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731583348.613598    2869 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-14 19:22:28.856183: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To en

In [ ]:
###try extract features with svd
import numpy as np

def extract_features_optimized(H, n_components=50):
    """
    Optimized feature extraction from complex channel matrix H.

    Parameters:
        H (numpy.ndarray): Complex channel matrix with shape (samples, 64, 2, 408).
        n_components (int): Number of dimensions to reduce to using SVD.

    Returns:
        numpy.ndarray: Reduced feature representation (samples, n_components).
    """
    num_samples, num_antennas, num_user_antennas, num_subcarriers = H.shape

    # Step 1: Compute spatial covariance matrix using truncated representation
    spatial_features = []
    for sample in H:
        reshaped_sample = sample.reshape(num_antennas, -1)  # (64, 816)
        spatial_cov = np.matmul(reshaped_sample, reshaped_sample.conj().T)  # (64, 64)
        # Use only the diagonal and few important off-diagonal elements
        spatial_diag = np.diag(spatial_cov)  # Main diagonal (64,)
        spatial_top_off_diag = np.mean(spatial_cov[:, :8], axis=1)  # Important off-diagonal elements
        spatial_features.append(np.concatenate((spatial_diag, spatial_top_off_diag)))  # (64 + 8,)

    spatial_features = np.array(spatial_features)  # Shape: (samples, 72)

    # Step 2: Compute frequency domain features (simple average and variance)
    frequency_features = []
    for sample in H:
        freq_avg = np.mean(np.abs(sample), axis=(0, 1))  # Average amplitude for each subcarrier (408,)
        freq_var = np.var(np.abs(sample), axis=(0, 1))  # Variance for each subcarrier (408,)
        frequency_features.append(np.concatenate((freq_avg, freq_var)))

    frequency_features = np.array(frequency_features)  # Shape: (samples, 816)

    # Step 3: Combine spatial and frequency features
    combined_features = np.hstack((spatial_features, frequency_features))  # Shape: (samples, 888)

    # Step 4: Dimensionality reduction using SVD
    U, S, Vt = np.linalg.svd(combined_features, full_matrices=False)
    reduced_features = U[:, :n_components] * S[:n_components]  # Project to top n_components

    return reduced_features


In [3]:
test_reduced = extract_features(H)

In [4]:
import numpy as np

def convert_to_real_amplitude_phase(H):
    """
    Convert complex data to a real-valued matrix using amplitude and phase.
    
    Parameters:
        H (numpy.ndarray): Complex-valued matrix with shape (samples, features).
        
    Returns:
        numpy.ndarray: Real-valued matrix with shape (samples, features * 2).
    """
    amplitude = np.abs(H)  # 振幅
    phase = np.angle(H)    # 相位
    return np.hstack((amplitude, phase))

# 使用示例
real_features = convert_to_real_amplitude_phase(test_reduced)


In [ ]:
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimension_tsne(features, n_components=2, batch_size=1000):
    """
    使用 t-SNE 将高维数据降维到 2D。
    
    Parameters:
        features (numpy.ndarray): 高维特征矩阵，形状为 (samples, n_features)。
        n_components (int): 降维后的目标维度，默认为2。
        
    Returns:
        numpy.ndarray: 降维后的坐标，形状为 (samples, n_components)。
    """
    tsne = TSNE(n_components=n_components, random_state=0, perplexity=50, n_iter=1000)
    low_dim_coords = tsne.fit_transform(features)
    return low_dim_coords

In [6]:
low_dim_coords = reduce_dimension_tsne(real_features)

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_t_sne.py:1162: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


In [12]:
csi_file = './low_dim_coords.npy'
np.save(csi_file, low_dim_coords)

In [7]:
test2_reduced = extract_features(H)

In [7]:
import numpy as np
def read_anch_file(file_path, anch_samp_num):

    anch_pos = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        line_fmt = [line.rstrip('\n').split(' ') for line in lines]
    for line in line_fmt:
        tmp = np.array([int(line[0]), float(line[1]), float(line[2])])
        if np.size(anch_pos) == 0:
            anch_pos = tmp
        else:
            anch_pos = np.vstack((anch_pos, tmp))
    return anch_pos

anch_pos_path = '../Dataset0/Dataset0InputPos3.txt'
anch_pos = read_anch_file(anch_pos_path, 2000)

In [4]:
low_dim_coords = np.load("/data/code/low_dim_coords.npy")

In [8]:
# 提取索引和真实坐标
train_indices = anch_pos[:, 0].astype(int)  # 索引 (2000,)
y_train = anch_pos[:, 1:]  # 真实坐标 (2000, 2)

# 按索引提取2000个数据
X_train = low_dim_coords[train_indices]

# 提取剩下的18000个数据
mask = np.ones(low_dim_coords.shape[0], dtype=bool)
mask[train_indices] = False
X_test = low_dim_coords[mask]

In [15]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
from sklearn.model_selection import train_test_split

# 标准化 X 和 y
scaler_X = StandardScaler()
X_train_std = scaler_X.fit_transform(X_train)
X_test_std = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_std = scaler_y.fit_transform(y_train)

# 将部分数据作为验证集
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train_std, y_train_std, test_size=0.15, random_state=21)

# 定义神经网络模型
model = Sequential([
    Dense(64, activation='relu', input_shape=(2,)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(2)  # 输出层
])

# 使用 Adam 编译模型
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# 使用 EarlyStopping 和 ReduceLROnPlateau 调整学习率并防止过拟合
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, verbose=1)

# 训练模型
model.fit(X_train_split, y_train_split, validation_data=(X_val, y_val), epochs=200, batch_size=32, verbose=1, callbacks=[early_stopping, reduce_lr])

# 预测并逆标准化
predict_midnn_std = model.predict(X_test_std)
predict_midnn_coords = scaler_y.inverse_transform(predict_midnn_std)


Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 1.0182 - val_loss: 1.0022 - learning_rate: 0.0010
Epoch 2/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.0027 - val_loss: 0.9999 - learning_rate: 0.0010
Epoch 3/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0047 - val_loss: 0.9988 - learning_rate: 0.0010
Epoch 4/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0135 - val_loss: 0.9954 - learning_rate: 0.0010
Epoch 5/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0194 - val_loss: 0.9967 - learning_rate: 0.0010
Epoch 6/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0089 - val_loss: 0.9963 - learning_rate: 0.0010
Epoch 7/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9900 - val_loss: 1.0022 - learning_rate: 0.0010
Epoch 8/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9924 - val_loss: 0.9993 - learning_rate: 0.0010
Epoch 9/200
40/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9856 
Epoch 9: ReduceLROnPlateau reducing learning rate to 0.000500000023748725

In [8]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from sklearn.model_selection import train_test_split

# 标准化 X 和 y
scaler_X = StandardScaler()
X_train_std = scaler_X.fit_transform(X_train)
X_test_std = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_std = scaler_y.fit_transform(y_train)

# 将部分数据作为验证集
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train_std, y_train_std, test_size=0.1, random_state=42)

# 定义神经网络模型
model = Sequential([
    Dense(128, input_shape=(2,)),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),
    Dense(256),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),
    Dense(128),
    LeakyReLU(alpha=0.1),
    Dropout(0.2),
    Dense(2)  # 输出层
])

# 使用 RMSprop 编译模型
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='mse')

# 使用 EarlyStopping 防止过拟合
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 训练模型
model.fit(X_train_split, y_train_split, validation_data=(X_val, y_val), epochs=300, batch_size=64, verbose=1, callbacks=[early_stopping])

# 预测并逆标准化
predict_midnn_std = model.predict(X_test_std)
predict_midnn_coords = scaler_y.inverse_transform(predict_midnn_std)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
W0000 00:00:1731583650.998235    2869 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.9931 - val_loss: 1.0265
Epoch 2/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9807 - val_loss: 1.0273
Epoch 3/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.0053 - val_loss: 1.0269
Epoch 4/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9868 - val_loss: 1.0277
Epoch 5/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0231 - val_loss: 1.0273
Epoch 6/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9867 - val_loss: 1.0297
Epoch 7/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9944 - val_loss: 1.0303
Epoch 8/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.0037 - val_loss: 1.0283
Epoch 9/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.9783 - val_loss: 1.0284
Epoch 10/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0004 - val_loss: 1.0297
Epoch 11/300
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0074 - val_loss: 1.0294
563/563 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [22]:
# 步骤 1: 合并锚点数据和预测值
# 创建一个占位矩阵用于保存最终结果
predicted_coords = np.zeros((20000, 2))
# indices 错位
adjusted_train_indices = train_indices - 1
predicted_coords[adjusted_train_indices] = y_train

# 获取剩余18000个数据的索引
remaining_indices = np.setdiff1d(np.arange(20000), adjusted_train_indices)
# 将预测值放入剩余位置
predicted_coords[remaining_indices] = predict_midnn_coords

In [23]:
data_truth = np.loadtxt('../Dataset0/Dataset0GroundTruth3.txt', dtype=float)

In [24]:
# 步骤 2: 与真实数据比较计算准确率
# 设定误差阈值，例如：欧氏距离小于0.05视为预测正确
threshold = 0.05
# 计算每个点的预测误差
errors = np.linalg.norm(predicted_coords - data_truth, axis=1)  # 欧氏距离
# 判断哪些点预测正确
correct_predictions = errors < threshold
accuracy = np.sum(correct_predictions) / 20000  # 准确率

# 输出结果
print("Accuracy: {:.2%}".format(accuracy))

Accuracy: 10.00%


In [25]:
# 计算每个点的预测误差（欧氏距离）
errors = np.linalg.norm(predicted_coords - data_truth, axis=1)  # 欧氏距离

# 计算欧氏距离的平均值
mean_error = np.mean(errors)

# 输出平均误差
print("Mean Euclidean Distance Error: {:.4f}".format(mean_error))


Mean Euclidean Distance Error: 84.6671


In [32]:
import numpy as np

def calculate_mean_euclidean_error(predicted_file, groundtruth_file):
    # 读取预测数据和真实数据
    predicted_coords = np.loadtxt(predicted_file)  # 假设文件格式为两列 x y 坐标
    data_truth = np.loadtxt(groundtruth_file)

    # 计算每个点的预测误差（欧氏距离）
    errors = np.linalg.norm(predicted_coords - data_truth, axis=1)
    
    # 计算欧氏距离的平均值
    mean_error = np.mean(errors)
    return mean_error

# 文件名
output_files = ["/data/Dataset0/Dataset0OutputPos1.txt", "/data/Dataset0/Dataset0OutputPos2.txt", "/data/Dataset0/Dataset0OutputPos3.txt"]
groundtruth_files = ["/data/Dataset0/Dataset0GroundTruth1.txt", "/data/Dataset0/Dataset0GroundTruth2.txt", "/data/Dataset0/Dataset0GroundTruth3.txt"]

# 计算每组的平均误差
mean_errors = [calculate_mean_euclidean_error(output, groundtruth) 
               for output, groundtruth in zip(output_files, groundtruth_files)]

# 计算所有数据的总平均误差
total_mean_error = np.mean(mean_errors)

# 输出结果
for i, error in enumerate(mean_errors, 1):
    print(f"Mean Euclidean Distance Error for set {i}: {error:.4f}")
print(f"Total Mean Euclidean Distance Error: {total_mean_error:.4f}")


Mean Euclidean Distance Error for set 1: 66.9143
Mean Euclidean Distance Error for set 2: 74.9610
Mean Euclidean Distance Error for set 3: 84.5370
Total Mean Euclidean Distance Error: 75.4708
